In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageTk
import csv
import pyautogui
from pytesseract import pytesseract
from pytesseract import Output
import tkinter as tk
from tkinter import simpledialog
from tkinter import messagebox
from tkinter import filedialog as fd
Image.MAX_IMAGE_PIXELS = 1000000000
pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
def show_image(x):
    screen_width, screen_height = pyautogui.size()
    if len(x.shape) == 3:
        (image_height, image_width,_) = x.shape
    else:
        (image_height, image_width) = x.shape
    resized_metric = .9
    while True:
        if(image_height < screen_height and image_width < screen_width):
            break
        image_width = int(image_width * resized_metric)
        image_height = int(image_height * resized_metric)   
        x = cv2.resize(x,(image_width,image_height))
    cv2.imshow("Matched image", x)
    cv2.waitKey(0)
    cv2.destroyWindow("Matched image")  
    return x

In [3]:
def scale_to_full_screen(image):
    screen_width, screen_height = pyautogui.size()
    if len(image.shape) == 3:
        (image_height, image_width,_) = image.shape
    else:
        (image_height, image_width) = image.shape
    resized_metric = .9
    while True:
        if(image_height < screen_height and image_width < screen_width):
            break
        image_width = int(image_width * resized_metric)
        image_height = int(image_height * resized_metric)   
        image = cv2.resize(image,(image_width,image_height))
    return image_width, image_height

In [4]:
def load_image():

    root = tk.Tk()
    root.title('Tkinter Open File Dialog')
    root.resizable(False, False)
    root.geometry('600x400')
    input_text1 = tk.StringVar()
    input_text2 = tk.StringVar()
    input_text3 = tk.StringVar()
    input_text4 = tk.StringVar()
    
    def save():
                
        global img_gray, img_thresh, Building_type, Section, Floor, type_of_plan
                
        Building_type = input_text1.get()
        Section = input_text2.get()
        Floor = input_text3.get()
        type_of_plan = input_text4.get()
        img_gray = cv2.cvtColor(img_rgb,cv2.COLOR_BGR2GRAY)
        img_thresh = cv2.threshold(img_gray,np.percentile(img_gray,90),255,cv2.THRESH_BINARY)[1]
        root.destroy() 
        root.quit()     

    def select_file():
        
        global filename, img_rgb, scaled_img
        
        filetypes = [('image file', '*.bmp')]
        filename = fd.askopenfilename(title='Open a file', initialdir='/', filetypes=filetypes)
        tk.messagebox.showinfo(title='Selected File', message=filename)
        img_rgb = cv2.imread(filename)
        scaled_img = show_image(img_rgb)
        label11 = tk.Label(root, text = filename, font=('courier', 15, 'bold'))
        label11.grid(row = 1, column = 3, ipadx=10, ipady=10)
    
    label1 = tk.Label(root, text = "The file path:", font=('courier', 15, 'bold'))
    label1.grid(row = 1, column = 1, ipadx=10, ipady=10)
        
    open_button = tk.Button(root, text='Open a File', command=select_file)
    open_button.grid(row = 0, column = 2, pady=15)
    
    save_button = tk.Button(root, text='Save', command=save)
    save_button.grid(row = 6, column = 2)
    
    entry2 = tk.Entry(root, textvariable = input_text1, justify = 'center' , font=('courier', 10, 'normal'))
    entry2.grid(row = 2, column = 3, ipadx=10, ipady=10)
    
    label2 = tk.Label(root, text = "Building type:", font=('courier', 15, 'bold'))
    label2.grid(row = 2, column = 1, ipadx=10, ipady=10)
    
    entry3 = tk.Entry(root, textvariable = input_text2, justify = 'center' , font=('courier', 10, 'normal'))
    entry3.grid(row = 3, column = 3, ipadx=10, ipady=10)
    
    label3 = tk.Label(root, text = "Section:", font=('courier', 15, 'bold'))
    label3.grid(row = 3, column = 1, ipadx=10, ipady=10)

    entry4 = tk.Entry(root, textvariable = input_text3, justify = 'center' , font=('courier', 10, 'normal'))
    entry4.grid(row = 4, column = 3, ipadx=10, ipady=10)
    
    label4 = tk.Label(root, text = "Floor:", font=('courier', 15, 'bold'))
    label4.grid(row = 4, column = 1, ipadx=10, ipady=10)
    
    entry5 = tk.Entry(root, textvariable = input_text4, justify = 'center' , font=('courier', 10, 'normal'))
    entry5.grid(row = 5, column = 3, ipadx=10, ipady=10)
    
    label5 = tk.Label(root, text = "Type of plan:", font=('courier', 15, 'bold'))
    label5.grid(row = 5, column = 1, ipadx=10, ipady=10)

    root.mainloop()

In [5]:
def gammaCorrection(src, gamma):
    
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)
    
    return cv2.LUT(src, table)

In [6]:
def find_legend(legend_images):
    
    templates = {}
    templates['legend'] = legend_images
    for legend_image in legend_images:
        legend_gray = cv2.cvtColor(legend_image,cv2.COLOR_BGR2GRAY)
        legend_image_edges = cv2.Canny(legend_gray,50,150)
        lines = cv2.HoughLinesP(legend_image_edges, 1, np.pi/180, threshold = 2, minLineLength = 3, maxLineGap = 3)
        lines = np.resize(lines,[lines.shape[0],lines.shape[2]])
        x1_start,y1_start,x2_start,y2_start = np.min(lines, axis = 0)
        x_start = min(x1_start,x2_start)
        y_start = min(y1_start,y2_start)
        x1_end,_,x2_end,_ = np.max(lines, axis = 0)
        x_end = max(x1_end,x2_end)
        lines = np.delete(lines,[0,2],1)
        lines = np.sort(lines, axis = 1)
        lines = np.sort(lines[:,0])
        for j in range(len(lines)-1):
            if lines[j+1]-lines[j] > 15:
                new_template = legend_image[y_start-15:lines[j]+15,x_start:x_end]
                add_new_template(templates, new_template, legend_image, [y_start, lines[j], x_start, x_end])
                y_start = lines[j+1]
            elif j == len(lines)-2:
                new_template = legend_image[y_start-15:lines[j]+20,x_start:x_end]
                add_new_template(templates, new_template, legend_image, [y_start, lines[j], x_start, x_end])
            
    return templates

In [7]:
def add_new_template(templates_dic, template, legend_image, template_location):

    root = tk.Tk()
        
    def save():  
                        
        file_name = entry1.get()
        if file_name == 'cancel' or file_name == ' cancel' or file_name == 'בטל':
            root.destroy() 
            root.quit() 
        elif file_name in templates_dic:
            creat_info_box("Template name already exists, choose other name")
        else:
            templates_dic[file_name] = template
            root.destroy() 
            root.quit() 
            
    def skip():
        
        root.destroy() 
        root.quit() 
        
    def edit():
        
        l = template_location
        template_area_in_img = legend_image[l[0]-15:l[1]+100, l[2]:l[3]]
        resize_width, resize_height = scale_to_full_screen(template_area_in_img)
        edited_template = crop_and_edit(template_area_in_img,
                                        cv2.resize(template_area_in_img,(resize_width,resize_height)),
                                        'template')
        add_new_template(templates_dic, edited_template, legend_image, template_location)
        root.destroy() 
        root.quit()
        
    root.geometry('400x300')
    input_text = tk.StringVar()
    
    bottom = tk.Frame(root)
    bottom.pack(side='bottom', fill='both')
    
    edit = tk.Button(root, text='Edit', command = edit)
    edit.pack(in_ = bottom, side = 'left', padx= 30, pady=10)
    
    save = tk.Button(root, text='Save', command = save)
    save.pack(in_ = bottom, side = 'left', padx= 75, pady=10)  
    
    skip = tk.Button(root, text='Skip', command = skip)
    skip.pack(in_ = bottom, side = 'right', padx= 30, pady=10)
    
    entry1 = tk.Entry(root, textvariable = input_text, justify = 'center' , font=('courier', 15, 'normal'))
    entry1.pack(side = 'bottom', ipadx=10, ipady=10)
    entry1.focus_force()

    name_label = tk.Label(root, text = 'Enter template name:', font=('courier', 15, 'bold'))
    name_label.pack(side = 'bottom', ipadx=3, ipady=10)

    photo = ImageTk.PhotoImage(image = Image.fromarray(template), master = root)
    photo_label = tk.Label(root, image=photo)
    photo_label.pack(side = 'bottom' ,anchor='n')
    
    root.mainloop() 

In [8]:
def crop_and_edit(real_size_image, fit_screen_image, name_of_image):
    
    global image, resize_scale, real_size_img, original_room, refPt, mask_cord, cropped_room
    
    finish = 0
    rooms = {}
    image = fit_screen_image.copy()
    real_size_img = real_size_image.copy()
    resize_scale = real_size_image.shape[0]/fit_screen_image.shape[0]
    if name_of_image == 'rooms':
        counter = 0
    else:
        counter = 3
    while finish == 0:
        if counter == 0:
            creat_info_box('Crop the legend')
        elif counter == 1:
            creat_info_box('Crop the entire floor')
        elif counter == 2:
            cv2.destroyAllWindows()
            image = show_image(original_room)
            if name_of_image == 'rooms':
                creat_info_box('Crop the rooms')
                resize_scale = rooms['the entire floor'][0].shape[0]/image.shape[0]
            real_size_img = original_room.copy()
            
        refPt = []
        edited_img = False
        cropping = False
        clone = image.copy()
        cv2.namedWindow("image")
        cv2.setMouseCallback("image", click_and_crop)
    
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("r"):
                image = clone.copy()
            elif key == ord("q"):
                finish = 1
                break
            elif key == ord("c"):
                counter += 1
                break
            elif key == ord("e"):
                edited_img = True
                creat_info_box('Crop the area you would like to erase')
                if len(refPt) < 2:
                    creat_info_box('Image was not marked')
                    pass
                
                mask_cord = []
                size = resize_scale
                cropped_room = real_size_img[round(refPt[0][1]*size):round(refPt[1][1]*size),
                                             round(refPt[0][0]*size):round(refPt[1][0]*size)]
                resize_width, resize_height = scale_to_full_screen(cropped_room)
                editing = False
                finish_edit = False
                cv2.namedWindow("editing mode")
            
                while finish_edit == False:
                    cv2.setMouseCallback("editing mode", click_and_mask)
                    editing_img_clone = cropped_room.copy()
                    while True:
                        cv2.imshow("editing mode", cv2.resize(editing_img_clone, (resize_width, resize_height)))
                        key = cv2.waitKey(1) & 0xFF
                        if key == ord("r"):
                            cropped_room = editing_img_clone.copy()
                            cv2.imshow("editing mode", cv2.resize(cropped_room, (resize_width, resize_height)))
                            break
                        elif key == ord("q"):
                            finish_edit = True
                            cv2.destroyWindow("editing mode")
                            break
                        elif key == ord("e"):
                            cv2.imshow("editing mode", cv2.resize(cropped_room, (resize_width, resize_height)))
                            editing_img_clone = cropped_room
                            break
                counter += 1
                break
        if finish == 1:
            break
        if len(refPt) == 2:
            if edited_img == True:
                original_room = cropped_room
            else:
                size = resize_scale
                original_room = real_size_img[round(refPt[0][1]*size):round(refPt[1][1]*size),
                                              round(refPt[0][0]*size):round(refPt[1][0]*size)]
            if name_of_image != 'rooms':
                cv2.destroyAllWindows()
                return original_room    
            ROOT = tk.Tk()
            ROOT.withdraw()
            if counter == 1:
                USER_ANS = messagebox.askyesno('Cropping mode',
                                               "The legend cropped. Are there any more parts to the legend?")
                if USER_ANS == True:
                    counter = 0
                if len(rooms) == 0:
                    rooms['legend'] = [original_room]
                else:
                    rooms['legend'].append(original_room)
            elif counter == 2:
                rooms['the entire floor'] = [original_room]
                creat_info_box('The entire floor cropped')
            else:
                USER_INP = simpledialog.askstring(title="Room number", prompt="What's the room number?")
                if USER_INP in rooms.keys():
                    USER_ANS = messagebox.askyesno("Room number",
                                                   "Room number already exists. Would you like to choose other number?")
                    if USER_ANS == True:
                        _ = show_image(original_room)
                        USER_INP = simpledialog.askstring(title="Room number", prompt="What's the room number?")
                    else:
                        USER_ANS = messagebox.askyesno("Old room number",
                                                       "Would you like to change the number of the existing room?")
                        if USER_ANS == True:
                            _ = show_image(rooms[USER_INP][0])
                            NEW_USER_INP = simpledialog.askstring(title="Room number", prompt="What's the room number?")
                            rooms[NEW_USER_INP] = rooms.pop(USER_INP)
                        else:
                            USER_ANS = messagebox.askyesno("Question",
                                                           "Are you sure? If you wont change the old room number the new room will take it place")
                            if USER_ANS == False:
                                _ = show_image(rooms[USER_INP][0])
                                NEW_USER_INP = simpledialog.askstring(title="Room number", prompt="What's the room number?")
                                rooms[NEW_USER_INP] = rooms.pop(USER_INP)
                rooms[USER_INP] = [original_room]
        
    cv2.destroyAllWindows()
    return rooms

In [9]:
def crop_by_edges(x):
    templates = []
    for i in x:
        if i == 'legend':
            continue
        y = x[i]
        y_thresh = cv2.threshold(y,175,255,cv2.THRESH_BINARY)[1]
        y_edges = cv2.Canny(y_thresh,50,150)
        pts = np.argwhere(y_edges>0)
        y1,x1 = pts.min(axis=0)
        y2,x2 = pts.max(axis=0)
        cropped = y_thresh[y1:y2, x1:x2]
        templates.append(cropped)
    return templates

In [10]:
def click_and_crop(event, x, y, flags, param):

    global refPt, cropping, original_room
    
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True
    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        size = resize_scale
        original_room = real_size_img[round(refPt[0][1]*size):round(refPt[1][1]*size),
                                      round(refPt[0][0]*size):round(refPt[1][0]*size)]
        show_image(original_room)

In [11]:
def click_and_mask(event, x, y, flags, param):
    
    global mask_cord, editing, original_room, cropped_room
        
    if event == cv2.EVENT_LBUTTONDOWN:
        mask_cord = [(x, y)]
        editing = True
    elif event == cv2.EVENT_LBUTTONUP:
        mask_cord.append((x, y))
        editing = False
        cv2.destroyWindow("editing mode")
        resize_width, resize_height = scale_to_full_screen(cropped_room)
        size_width = cropped_room.shape[1]/resize_width
        size_height = cropped_room.shape[0]/resize_height
        original_room = real_size_img[round(refPt[0][1]*size_height):round(refPt[1][1]*size_height),
                                      round(refPt[0][0]*size_width):round(refPt[1][0]*size_width)]
        cropped_room[round(mask_cord[0][1]*size_height):round(mask_cord[1][1]*size_height),
                     round(mask_cord[0][0]*size_width):round(mask_cord[1][0]*size_width)] = 255
        show_image(cropped_room)

In [12]:
def creat_info_box(text):
    root = tk.Tk()
    root.withdraw()
    messagebox.showinfo("Message", text)
    root.destroy()

In [14]:
def choose_template_threshold(template):
    h, w = template.shape
    edges = cv2.Canny(template,70,150)
    lines = cv2.HoughLinesP(
        edges,
        1,
        np.pi/180,
        threshold = round(min(h,w)*0.5),
        minLineLength = 15,
        maxLineGap = 3
    )
    try:
        if len(lines) > 4:
            threshold = 0.65
        else:
            threshold = 0.6 + len(lines)*0.05
    except TypeError:
        threshold = 0.65
    return threshold

In [15]:
def split_to_tiles(img,height,width):
    width_cutoff = width // 2
    left1 = img[:, :width_cutoff]
    right1 = img[:, width_cutoff:]
    img = cv2.rotate(left1, cv2.ROTATE_90_CLOCKWISE)
    height = img.shape[0]
    width = img.shape[1]
    width_cutoff = width // 2
    l1 = img[:, :width_cutoff]
    l2 = img[:, width_cutoff:]
    l1 = cv2.rotate(l1, cv2.ROTATE_90_COUNTERCLOCKWISE)
    l2 = cv2.rotate(l2, cv2.ROTATE_90_COUNTERCLOCKWISE)
    img = cv2.rotate(right1, cv2.ROTATE_90_CLOCKWISE)
    height = img.shape[0]
    width = img.shape[1]
    width_cutoff = width // 2
    r1 = img[:, :width_cutoff]
    r2 = img[:, width_cutoff:]
    r1 = cv2.rotate(r1, cv2.ROTATE_90_COUNTERCLOCKWISE)
    r2 = cv2.rotate(r2, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return(l1,l2,r1,r2)

In [16]:
def rotate(templates):
    for i in range(len(templates)):
        templates[i] = cv2.rotate(templates[i],cv2.ROTATE_90_CLOCKWISE)
    return templates

In [17]:
def find_template_position(template, image, threshold):
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(res > threshold)
    h, w = template.shape
    l = []
    accuracy = []
    for pt in zip(*loc[::-1]):
        if len(l) == 0:
            l.append(pt)
            accuracy.append(res[(pt[1],pt[0])])
        else:
            for i in range (len(l)):
                if abs(l[i][0]-pt[0]) < h*0.9 and abs(l[i][1]-pt[1]) < w*0.9:
                    u = 1
                else:
                    u = 0 
                if u == 1:
                    if res[(pt[1],pt[0])] > accuracy[i]:
                        l.pop(i)
                        accuracy.pop(i)
                        l.append(pt)
                        accuracy.append(res[(pt[1],pt[0])])
                        break
                elif i == len(l)-1:
                    l.append(pt)
                    accuracy.append(res[(pt[1],pt[0])])                    
    return([l,accuracy])

In [18]:
def choose_wall(C,template, image):
    h_image, w_image = image.shape
    h, w = template.shape
    if  C[1] < h_image*0.05 and w_image*0.05 < C[0] < w_image*0.95 :
          Part_in_the_room = 'קיר אחורי'
    elif (C[1] + h) > h_image*0.95 and w_image*0.05 < C[0] < w_image*0.95:
          Part_in_the_room = 'קיר קדמי'
    elif (C[0] + w) > w_image*0.95 and h_image*0.05 < C[1] < h_image*0.95:
          Part_in_the_room = 'קיר שמאלי'
    elif C[0] < w_image*0.05  and h_image*0.05 < C[1] < h_image*0.95:
          Part_in_the_room = 'קיר ימני'
    else:
        Part_in_the_room = 'רצפה / תקרה'
    return Part_in_the_room

In [19]:
def creat_electric_inspaction(Building_type,Section,Floor,Room_number,Zone,Part_in_the_room,
                              Description,Amount,Hight,Coordinate,accuracy,type_of_plan):
    return [{'Building type':Building_type,'Section':Section,'Floor':Floor,'Room':Room_number,'Zone':Zone,
             'Part in the room':Part_in_the_room,'Code':'חשמל-תשתיות','Description':Description,'Amount':Amount,
             'Hight':Hight,'Coordinate':Coordinate,'Accuracy':accuracy,'Type of plan':type_of_plan},
            {'Building type':Building_type,'Section':Section,'Floor':Floor,'Room':Room_number,'Zone':Zone,
             'Part in the room':Part_in_the_room,'Code':'חשמל-עבודות גמרים','Description':Description,'Amount':Amount,
             'Hight':Hight,'Coordinate':Coordinate,'Accuracy':accuracy,'Type of plan':type_of_plan}]

In [20]:
def write_to_dict(image,templates,template_name,location_dict,room_number):
    Amount = 1
    for i in range(len(templates)):
        try:
            h, w = templates[i].shape
        except ValueError:
            templates[i] = cv2.cvtColor(templates[i], cv2.COLOR_BGR2GRAY)
            h, w = templates[i].shape
        L1,L2,R1,R2 = split_to_tiles(templates[i],h,w)
        threshold = choose_template_threshold(templates[i])
        y = find_template_position(templates[i],image,threshold)
        for j in range(len(y[0])):
            cropped_image = image[y[0][j][1]:y[0][j][1]+h, y[0][j][0]:y[0][j][0]+w]
            if len(find_template_position(L1,cropped_image,threshold*0.7)[0])==0 \
            or len(find_template_position(L2,cropped_image,threshold*0.7)[0])==0 \
            or len(find_template_position(R1,cropped_image,threshold*0.7)[0])==0 \
            or len(find_template_position(R2,cropped_image,threshold*0.7)[0])==0:
                continue 
            Part_in_the_room = choose_wall(y[0][j], templates[i], image)
            new_inspaction = creat_electric_inspaction(Building_type,Section,Floor,room_number,'חדר',
                                                       Part_in_the_room,template_name[i],Amount,0.9,y[0][j],
                                                       y[1][j],type_of_plan)
            if len(location_dict) == 0:
                location_dict[y[0][j]] = new_inspaction
            else:
                x = list(location_dict.keys()) 
                for location in range(len(x)):        
                    if abs(y[0][j][0]-x[location][0])<w*0.5 and abs(y[0][j][1]-x[location][1])<h*0.5:
                        if location_dict[x[location]][0]['Accuracy'] < new_inspaction[0]['Accuracy']:
                            location_dict[y[0][j]] = new_inspaction
                            location_dict.pop(x[location])
                        break
                    elif location == len(x)-1:
                        location_dict[y[0][j]] = new_inspaction
            cv2.rectangle(image, y[0][j], (y[0][j][0] + w, y[0][j][1] + h), (0,0,255), 2) 
    return location_dict

In [21]:
def find_doubles(image, location_dict):
    image_data = pytesseract.image_to_boxes(image, lang = 'eng', output_type = Output.DICT,
                                            config= r'-c tessedit_char_whitelist=xX0123456789 --psm 6')

    num2word = {'2': ' כפול',
                '3': ' משולש',
                '4': ' מרובע',
                '5': ' חמישה',
                '6': ' שישיה',
                '7': ' שביעיה',
                '8': ' שמיניה',
                '9': ' תשיעיה',
                '10': ' עשיריה'}
    
    location_list = list(location_dict.keys())
    locations = np.array(location_list)
    
    if len(image_data) == 0 or len(location_list) == 0:
        return None
    
    for i in range(len(image_data['char'])-1):
        if image_data['char'][i].lower() == 'x' and image_data['char'][i+1] in list(num2word.keys()):
            x,y = image_data['left'][i], image_data['top'][i]
            point = np.array((x,y))
            distances = np.linalg.norm(locations - point)
            min_index = np.argmin(distances)
            index = location_list[min_index]
            if location_dict[index][0]['Amount'] != 1:
                location_dict[index][0]['Description'] = location_dict[index][0]['Description'][:-len(num2word[location_dict[index][0]['Amount']])]
                location_dict[index][1]['Description'] = location_dict[index][1]['Description'][:-len(num2word[location_dict[index][1]['Amount']])]
            location_dict[index][0]['Description'] = location_dict[index][0]['Description'] + num2word[image_data['char'][i+1]]
            location_dict[index][0]['Amount'] = image_data['char'][i+1]
            location_dict[index][1]['Description'] = location_dict[index][1]['Description'] + num2word[image_data['char'][i+1]]
            location_dict[index][1]['Amount'] = image_data['char'][i+1]
            np.delete(locations,min_index,0)
            del location_list[min_index]


In [22]:
def find_height(image, location_dict):
    image_data = pytesseract.image_to_boxes(image, lang = 'eng', output_type = Output.DICT,
                                            config= r'-c tessedit_char_whitelist=H0123456789=. --psm 6')
    location_list = list(location_dict.keys())
    locations = np.array(location_list)
    
    if len(image_data) == 0 or len(location_list) == 0:
        return None
    
    for i in range(len(image_data['char'])-3):
        if image_data['char'][i] == 'H' and image_data['char'][i+1] == '=':
            x,y = image_data['left'][i],image_data['top'][i]
            point = np.array((x,y))
            distances = np.linalg.norm(locations - point)
            min_index = np.argmin(distances)
            if min_index < len(image_data['char'])-5:
                 if image_data['char'][i+3] == '.':
                            location_dict[location_list[min_index]][0]['Hight'] = image_data['char'][i+2]\
                                                                                + image_data['char'][i+3]\
                                                                                + image_data['char'][i+4]\
                                                                                + image_data['char'][i+5]
                            location_dict[location_list[min_index]][1]['Hight'] = image_data['char'][i+2]\
                                                                                + image_data['char'][i+3]\
                                                                                + image_data['char'][i+4]\
                                                                                + image_data['char'][i+5]
            else:
                location_dict[location_list[min_index]][0]['Hight'] = image_data['char'][i+2]
                location_dict[location_list[min_index]][1]['Hight'] = image_data['char'][i+2]  
        
            np.delete(locations,min_index,0)
            del location_list[min_index]

In [23]:
def write_to_csv(file_name, info):
    with open(file_name, 'a+', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=field_names)
        for i in info:
            writer.writerows(i)

# Uplode BMP file

In [24]:
load_image()

# Crop rooms

In [25]:
rooms = crop_and_edit(img_rgb, scaled_img, 'rooms')

# Finding templates in legend

In [26]:
templates = find_legend(rooms['legend'])

# Creat the dict and the csv file

In [27]:
field_names = ['inspection number','Building type','Section','Floor','Room','Zone','Part in the room','Code','Description',
              'Amount','Hight','Coordinate','Accuracy','Type of plan']

root = tk.Tk()
root.geometry('530x170')
input_text = tk.StringVar()

def save():
    
    global output_file
    
    output_file = entry1.get()
    root.destroy() 
    root.quit()

label1 = tk.Label(root, text = "Enter the name of the output file:", font=('courier', 15, 'bold'))
label1.grid(row = 0, ipadx=10, ipady=10)
    
entry1 = tk.Entry(root, textvariable = input_text, justify = 'center' , font=('courier', 15, 'normal'))
entry1.grid(row = 1, ipadx=10, ipady=10)

save = tk.Button(root, text='Save', command = save)
save.grid(row = 2, padx= 75, pady=10) 

root.mainloop()

with open('{}.csv'.format(output_file), 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()

# Electric

In [28]:
clean_templates = crop_by_edges(templates)
template_name = list(templates.keys())
template_name.remove('legend')
for room_number in rooms:
    if room_number not in ['legend', 'the entire floor','zoom']:
        location_dict = {}
        for i in range(4):
            gamma_correct_room = gammaCorrection(rooms[room_number][0] ,2)
            _ , room_thresh = cv2.threshold(cv2.cvtColor(gamma_correct_room ,cv2.COLOR_BGR2GRAY),
                                            np.percentile(img_gray, 5),255,cv2.THRESH_BINARY)
            if i == 0:
                location_dict = write_to_dict(room_thresh,clean_templates,template_name,location_dict,room_number)
            elif i == 1:
                location_dict = write_to_dict(room_thresh,rotate(clean_templates),template_name,location_dict,room_number)
            elif i == 2:
                location_dict = write_to_dict(room_thresh,rotate(clean_templates),template_name,location_dict,room_number)
            elif i == 3:
                location_dict = write_to_dict(room_thresh,rotate(clean_templates),template_name,location_dict,room_number)
                find_height(room_thresh, location_dict)
                find_doubles(room_thresh, location_dict)
                rooms[room_number].append(location_dict.copy()) 
        write_to_csv('{}.csv'.format(output_file),list(location_dict.values()))

In [29]:
df = pd.read_csv('{}.csv'.format(output_file), encoding = "ISO-8859-8")
walls = ['קיר ימני','קיר קדמי','קיר שמאלי','קיר אחורי','רצפה / תקרה']
df['Part in the room'] = pd.Categorical(df['Part in the room'], categories = walls)
df_sorted = df.sort_values(by=['Room', 'Part in the room', 'Coordinate'])
df_sorted['inspection number']= df_sorted.groupby(['Room']).cumcount()
df_sorted.to_csv('{}.csv'.format(output_file), index=False ,encoding = 'utf-8-sig')

In [30]:
creat_info_box('File saved!')